In [ ]:
import geopandas as gpd
import yaml
import subprocess

## Create configurations

In [ ]:
%%bash
mkdir ./config

In [ ]:
root_path = "/home/notebook/shared-seabee-ns9879k"
model = "detection/2022_nina_birds_20230817"
workdir = "./nrdata"

`hub.yaml` configures the program and paths for the hub. We don't have a gpu on the hub and data is mounted so we can skip minio.

In [ ]:
device = "cpu"
with open("./config/hub.yaml", "w") as f:
    yaml.dump(
        {
            "DPATH_WORK": f"{workdir}/work",
            "DPATH_MODELS": f"{root_path}/models",
            "DPATH_RESULTS": f"{workdir}/results",
            "DPATH_PRETRAINED": f"{workdir}/pretrained",
            "MINIO": {"USE": False},
            "TEST": {"DEVICE": device},
        },
        f,
    )

In [ ]:
def dump_image_detection_config(orthophoto_file: str, root_path: str, model: str):
    """`image_detection.yaml` configures the nr program for training or detection on various images"""

    with open("./config/image_detection.yaml", "w") as f:
        yaml.dump(
            {
                "mode": "production",
                "task": "detection",
                "score_threshold": 0.1,
                "model": {"id": model},
                "dataset": {
                    "root": root_path,
                    "annotations": {
                        "crs": "epsg:4326",
                        "column_main_class": "species",
                        "columns_subtasks": ["activity", "sex", "age"],
                    },
                    "test_filenames": [orthophoto_file],
                },
            },
            f,
        )


def run_detection(mission_name: str):
    """Run the detection process elements

    A detection run consist of:
        1. preprocess the images
        2. the main detection run
        3. post process generating the gpkg data product

    """
    subprocess.run(
        [
            "nrseabee_det_preproc",
            "-c",
            "config/hub.yaml",
            "-o",
            "config/nrseabee_det_preproc.yaml",
            "config/image_detection.yaml",
        ],
        capture_output=True,
    )
    # start the detection run, (this can take some time)
    subprocess.run(
        [
            "nrseabee_det_test",
            "-c",
            "config/hub.yaml",
            "-o",
            "config/nrseabee_det_test.yaml",
            "config/nrseabee_det_preproc.yaml",
        ],
        capture_output=True,
    )
    # generate the geopackage with detections
    subprocess.run(
        [
            "nrseabee_det_postproc",
            "-c",
            "config/hub.yaml",
            "-o",
            f"config/{mission_name}_postproc.yaml",
            "config/nrseabee_det_test.yaml",
        ],
        capture_output=True,
    )

In [ ]:
mission_name = "oslofjorden_lyseren-zoom_202306271117"
orthophoto_file = f"seabirds/2023/{mission_name}/orthophoto/{mission_name}.tif"

mission_list = [mission_name]

## Run the bird detection

1. 

In [ ]:
for mission_name in mission_list:
    run_detection(mission_name)